# Testing module

In [1]:
import csv
import os
import pandas as pd
import pickle 
import numpy as np
import mediapipe as mp
import cv2

In [2]:
model = pickle.load(open('model.pkl', 'rb'))

In [3]:
mpDraw= mp.solutions.drawing_utils
mpPose = mp.solutions.pose
pose = mpPose.Pose()

In [ ]:
cap = cv2.VideoCapture(0)
mode = 0
correct = 0
count = 0
direc = 0
dcount = 30
ncount = 3
pcount = 1
show = 0
text = 'NEW'
w=260
while cap.isOpened():
    if show == 0:
        ret, frame = cap.read()
    if show == 1:
        frame = cv2.imread('1.jpg')
        if count > 9:
            w=210
        if count > 99:
            w=150
        cv2.putText(frame, str(int(count)), (w,280), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 255, 255), 8, cv2.LINE_AA)
    
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = mpPose.Pose().process(image)
        
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    try:
        poseX = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in poseX]).flatten())
        
        X = pd.DataFrame([pose_row])
        model_class = model.predict(X)[0]
        
        Y = pd.DataFrame([pose_row])
        model_class = model.predict(Y)[0]
        
        
        if model_class == 'exit':
            cv2.putText(image, str('STOP'), (240,210), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 5, cv2.LINE_AA)
            pcount -= 1
            if pcount == -5:
                show = 1
        
        cv2.rectangle(image,(150 ,0),(500,480),(0, 255, 0),3)

        if mode == 0:
            cv2.putText(image, str('READY?'), (210,250), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 4, cv2.LINE_AA)
            dcount = 30
            if model_class == 'up':
                mode = 1
        if mode == 1:
            dcount +=1
            cv2.putText(image, str(ncount), (275,250), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 255, 255), 5, cv2.LINE_AA)
            if dcount > 40:
                mode = 2
        if mode == 2:
            dcount +=1
            cv2.putText(image, str(ncount-1), (275,250), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 255, 255), 5, cv2.LINE_AA)
            if dcount > 50:
                mode = 3
        if mode == 3:
            dcount +=1
            cv2.putText(image, str(ncount-2), (275,250), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 255, 255), 5, cv2.LINE_AA)
            if dcount > 60:
                mode = 4
        if mode == 4:
            cv2.putText(image, str('Squats 1 time'), (200,60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str('Stay in your position'), (160,450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            if model_class == 'down':
                if direc == 0:
                    count += 0.5
                    direc = 1
            if model_class == 'up':
                if direc == 1:
                    count += 0.5
                    direc = 0
            if count == 1:
                text = 'PERFECT'
            if text == 'PERFECT':    
                cv2.putText(image, str(text), (190,250), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 5, cv2.LINE_AA)
                dcount +=1
                if dcount > 65:
                    count = 0
                    mode = 5
        if mode == 5:
            if model_class == 'down':
                if direc == 0:
                    count += 0.5
                    direc = 1
                    
            if model_class == 'up':
                if direc == 1:
                    count += 0.5
                    direc = 0
                    
            cv2.rectangle(image, (0,0), (100, 90), (37, 117, 242), -1)
                
            cv2.putText(image, 'COUNT'
                , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(int(count))
                , (20,70), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4, cv2.LINE_AA)
        
    except:
         pass
     
    cv2.imshow('Test', image)
        
    if cv2.waitKey(10) & 0XFF == ord('c'):
        break

cap.release()
cv2.destroyAllWindows()